# Lesson 5: Python on GPUs

Lets see what sort of GPU we have

In [1]:
!nvidia-smi

Sun Jun 15 12:28:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.51.03              Driver Version: 575.51.03      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     Off |   00000000:DA:00.0 Off |                  N/A |
| 28%   29C    P8              8W /  250W |       3MiB /  11264MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!numba -s

System info:
--------------------------------------------------------------------------------
__Time Stamp__
Report started (local time)                   : 2025-06-15 12:28:58.257960
UTC start time                                : 2025-06-15 12:28:58.257963
Running time (s)                              : 3.213224

__Hardware Information__
Machine                                       : x86_64
CPU Name                                      : skylake-avx512
CPU Count                                     : 48
Number of accessible CPUs                     : 48
List of accessible CPUs cores                 : 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
CFS Restrictions (CPUs worth of runtime)      : None

CPU Features                                  : 64bit adx aes avx avx2 avx512bw
                                                avx512cd avx512dq avx512f avx512vl
                                       

Ok - we have a GPU and it looks to be setup correctly. Lets try to use the CUPY library

In [4]:
import cupy as cp

In [5]:
a = cp.random.normal(0, 1, 5 * 1024)
a

array([ 1.63037085,  0.62022341,  0.23962623, ..., -0.19829212,
       -2.46828039, -0.32868927], shape=(5120,))

In [6]:
cp.sum(a.reshape(5, 1024), axis=1)

array([  8.05924927, -29.7449419 , -28.58452852,  39.92025844,
         5.960045  ])

And Numba

In [5]:
import numba as nb
import numba.cuda

In [6]:
@nb.cuda.jit
def zero_out(array):
    i = nb.cuda.grid(1)
    array[i] = 0

In [7]:
zero_out[1024, 5](a)

In [8]:
a

array([0., 0., 0., ..., 0., 0., 0.], shape=(5120,))



![](img/blocks-of-array.svg)

If the `size_of_array` doesn't fit neatly into an integer number of blocks, a few threads will be wasted. To make sure that they don't update uninitialized data, you usually need to check for an excess in the kernel function:

```cuda
__global__ some_function(float* array) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    if (index < size_of_array) {
        array[index] = ...;
    }
}
```


## CuPy

[CuPy](https://cupy.dev/) is a library with _mostly_ the same interface as NumPy, but all arrays are in GPU memory, rather than the motherboard's RAM. Converting a NumPy array into a CuPy array, and vice-versa, copies data from RAM to GPU global memory and back.

In [8]:
import numpy as np
import cupy as cp

In [9]:
array_in_ram = np.random.uniform(0, 1, 100000000)

In [10]:
array_on_gpu = cp.asarray(array_in_ram)

In [11]:
%%timeit -r1 -n10

array_in_ram[:] += 1

60.9 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


In [12]:
%%timeit -r1 -n10

array_on_gpu[:] += 1

2.15 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


It's important that the distinction is visible like this, since copying an array between RAM and the GPU is often more time-consuming than the computation itself.

In [13]:
%%timeit -r1 -n10

cp.asarray(array_in_ram)   # from RAM to GPU

134 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


In [14]:
%%timeit -r1 -n10

array_on_gpu.get()         # from GPU to RAM

145 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)


Thus, you'll want to keep the data on the device that is performing calculations for as long as possible. If data can be created on the GPU, such as random numbers for a Monte Carlo calculation, use CuPy's functions to do it.

In [15]:
array_on_gpu = cp.random.uniform(0, 1, 100000000)

We can see more of what's going on by running Nvidia's `nsys-ui` profiler on the following line:

In [16]:
array = cp.random.uniform(5, 10, 100000000)
array.sort()
array

array([5.        , 5.00000007, 5.0000002 , ..., 9.99999962, 9.99999966,
       9.9999998 ])

![](img/nsys-profiler-1.png){. width="100%"}

The CUDA kernels (blue) from the first line runs

* `cupy_random_x_mod_1`
* `cupy_scale`

to make the random numbers, followed by

* `DeviceRadixSortHistogramKernel`
* `DeviceRadixSortOnesweepKernel`

to sort the random numbers. Meanwhile on the CPU (green), Python waits for the result with a

* `cudaStreamSynchronize`

Since the GPU is a separate device from the CPU, it runs independently. If you don't ask for the result, the Python function can finish while the GPU calculation is still running. It's only when you want to print the result, copy it to a NumPy array, or get any element as a Python number that CuPy asks the GPU to "synchronize"—wait for computation to be finished.

Since the GPU runs independently, what should it do if it encounters an error part-way through processing? It can't raise a Python error (the Python function call has already finished), so it has to do something else instead. CuPy defines some results that would be errors in NumPy. For instance, this array-slice asks for elements beyond the end of the array:

In [17]:
array = np.array([0.0, 1.1, 2.2, 3.3, 4.4, 5.5])

array[np.array([2, 3, 5, 6, 7, 8])]

IndexError: index 6 is out of bounds for axis 0 with size 6

But CuPy returns a result by wrapping the indexes around to the beginning of the array (6 → 0, 7 → 1, 8 → 2):

In [18]:
array = cp.array([0.0, 1.1, 2.2, 3.3, 4.4, 5.5])

array[cp.array([2, 3, 5, 6, 7, 8])]

array([2.2, 3.3, 5.5, 0. , 1.1, 2.2])

So you may need to check the correctness of your code in NumPy before running it at high speed in CuPy.

Next, we should note that just replacing NumPy with CuPy has the same shortcoming as array-oriented Python versus compiled code: although it's a first (easy) step toward speeding up a computation, it's not the fastest possible because intermediate arrays have to be allocated in each step.

Remember how the quadratic formula

In [19]:
a = cp.random.uniform(5, 10, 1000000)
b = cp.random.uniform(10, 20, 1000000)
c = cp.random.uniform(-0.1, 0.1, 1000000)

(-b + cp.sqrt(b**2 - 4*a*c)) / (2*a)

array([ 0.00354629,  0.00108686, -0.0014943 , ...,  0.00375009,
        0.00123045, -0.00259124])

is roughly equivalent to

In [20]:
tmp1 = cp.negative(b)            # -b
tmp2 = cp.square(b)              # b**2
tmp3 = cp.multiply(4, a)         # 4*a
tmp4 = cp.multiply(tmp3, c)      # tmp3*c
del tmp3
tmp5 = cp.subtract(tmp2, tmp4)   # tmp2 - tmp4
del tmp2, tmp4
tmp6 = cp.sqrt(tmp5)             # sqrt(tmp5)
del tmp5
tmp7 = cp.add(tmp1, tmp6)        # tmp1 + tmp6
del tmp1, tmp6
tmp8 = cp.multiply(2, a)         # 2*a
np.divide(tmp7, tmp8)            # tmp7 / tmp8

array([ 0.00354629,  0.00108686, -0.0014943 , ...,  0.00375009,
        0.00123045, -0.00259124])

Here's what that looks like in the profiler:

![](img/nsys-profiler-2.png){. width="100%"}

Each of the

* `cupy_power__float64_float_float64`
* `cupy_multiply__float_float64_float64`
* `cupy_sqrt__float64_float64`
* `cupy_true_divide__float64_float64_float64`
* `cupy_multiply__float64_float64_float64`
* `cupy_subtract__float64_float64_float64`
* `cupy_add__float64_float64_float64`
* `cupy_negative__float64_float64`

kernels runs quickly, but there are time-gaps between them in which arrays are allocated and dynamic code decides what to do next. It would be faster if the whole quadratic formula were "fused" into a single kernel.

To support this, CuPy has a JIT compiler that lets you write C++ CUDA code. For instance, the worst part of the calculation above is using a general floating-point `power` function,

* `cupy_power__float64_float_float64`

to compute `power(b, 2)`, which should be just `b * b`. Let's define a better function for "raising to an integer power".

In [21]:
intpow = cp.ElementwiseKernel("float64 x, int64 n", "float64 out", '''
    out = 1.0;
    for (int i = 0;  i < n;  i++) {
        out *= x;
    }
''', "intpow")
intpow

In [22]:
intpow(b, 2)

array([349.69803979, 119.27597606, 300.9489732 , ..., 227.78363405,
       235.76110542, 178.84017007])

In [23]:
b**2

array([349.69803979, 119.27597606, 300.9489732 , ..., 227.78363405,
       235.76110542, 178.84017007])

We can also do this with the whole quadratic formula.

In [24]:
quadratic_formula = cp.ElementwiseKernel("float64 a, float64 b, float64 c", "float64 out", '''
    out = (-b + sqrt(b*b - 4*a*c)) / (2*a);
''', "quadratic_formula")

quadratic_formula(a, b, c)

array([ 0.00354629,  0.00108686, -0.0014943 , ...,  0.00375009,
        0.00123045, -0.00259124])

In [25]:
%%timeit -r1 -n1000

(-b + cp.sqrt(b**2 - 4*a*c)) / (2*a)

1.34 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1,000 loops each)


In [26]:
%%timeit -r1 -n1000

quadratic_formula(a, b, c)

163 μs ± 0 ns per loop (mean ± std. dev. of 1 run, 1,000 loops each)


Both of the above used [ElementwiseKernel](https://docs.cupy.dev/en/stable/user_guide/kernel.html), which is like a NumPy ufunc: they take arrays and apply the function to each element. We could also write a fully generic [RawKernel](https://docs.cupy.dev/en/stable/user_guide/kernel.html#raw-kernels), but then we'd have to manage the `threads_per_block` and `blocks_per_grid` manually.

In [27]:
quadratic_formula_raw = cp.RawKernel(r'''
extern "C" __global__
void quadratic_formula_raw(const double* a, const double* b, const double* c, int length, double* out) {
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < length) {
        out[i] = (-b[i] + sqrt(b[i]*b[i] - 4*a[i]*c[i])) / (2*a[i]);
    }
}
''', "quadratic_formula_raw")

out = cp.empty_like(a)

threads_per_block = 1024
blocks_per_grid = int(np.ceil(len(out) / 1024))

quadratic_formula_raw((blocks_per_grid,), (threads_per_block,), (a, b, c, len(out), out))

out

array([ 0.00354629,  0.00108686, -0.0014943 , ...,  0.00375009,
        0.00123045, -0.00259124])

Thus, CuPy is two things:

* an array library with the NumPy interface, but all arrays are on GPUs,
* a JIT compiler of CUDA kernels written in C++.

This situation is similar to writing custom C++ code with pybind11, rather than writing the kernel in Python.

## Numba

Just as Numba can JIT compile Python for CPUs, it can [JIT compile Python to CUDA](https://numba.readthedocs.io/en/stable/cuda/index.html).

Here's the quadratic formula as a single CUDA kernel without writing any C++.

In [28]:
import numba.cuda  # must be explicitly imported
import math        # note that Numba-CUDA requires math.*; you can't use np.*

In [29]:
@nb.cuda.jit
def quadratic_formula_numba_cuda(a, b, c, out):
    i = nb.cuda.grid(1)   # 1-dimensional
    if i < len(out):
        out[i] = (-b[i] + math.sqrt(b[i]**2 - 4*a[i]*c[i])) / (2*a[i])

out = cp.empty_like(a)

threads_per_block = 1024
blocks_per_grid = int(np.ceil(len(out) / 1024))

quadratic_formula_numba_cuda[blocks_per_grid, threads_per_block](a, b, c, out)

out

array([ 0.00354629,  0.00108686, -0.0014943 , ...,  0.00375009,
        0.00123045, -0.00259124])

Here's what it looks like in the profiler:

![](img/nsys-profiler-3.png){. width="100%"}

The name is a long random string, but it all runs in a short block of time.

Note that the structure of a `@nb.cuda.jit` compiled function is equivalent to a C++ CUDA function: all that has changed is the Python syntax and the names of some of the functions:

```cuda
extern "C" __global__
void quadratic_formula_raw(const double* a, const double* b, const double* c, int length, double* out) {
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < length) {
        out[i] = (-b[i] + sqrt(b[i]*b[i] - 4*a[i]*c[i])) / (2*a[i]);
    }
}

quadratic_formula_raw<<<blocks_per_grid, threads_per_block>>>(a, b, c, size_of_array, out);
```

versus

```python
@nb.cuda.jit
def quadratic_formula_numba_cuda(a, b, c, out):
    i = nb.cuda.grid(1)   # 1-dimensional
    if i < len(out):
        out[i] = (-b[i] + math.sqrt(b[i]**2 - 4*a[i]*c[i])) / (2*a[i])

quadratic_formula_numba_cuda[blocks_per_grid, threads_per_block](a, b, c, out)
```

You still have to choose an optimal `blocks_per_grid` and `threads_per_block`, and the kernel gets its thread index through a special function, [nb.cuda.grid](https://numba.readthedocs.io/en/stable/cuda-reference/kernel.html#numba.cuda.grid).

## JAX

Just as JAX inspects array-oriented code, identifies which functions can be fused, and JIT compiles the fused kernel for the CPU, it can do the same for GPUs. In fact, a JAX array has a `device` parameter that determines whether it will be in RAM or on the GPU.

Instead of performing the same demonstration with the quadratic formula, see `mandelbrot-on-all-accelerators.ipynb` in your `notebooks` folder. It takes a single calculation, which draws the Mandelbrot set, and accelerates it using all of the techniques we've discussed, including CuPy's RawKernel, Numba-CUDA, and JAX-CUDA. Here's the bottom line:

![](img/plot-mandelbrot-on-all-accelerators.svg){. width="100%"}

Custom CUDA kernels are the fastest, regardless of whether they are compiled by CuPy, Numba, or JAX.

## Awkward Array

Awkward Arrays can be copied to a GPU using [ak.to_backend](https://awkward-array.org/doc/main/reference/generated/ak.to_backend.html) with `backend="cuda"`.

In [30]:
import awkward as ak
import uproot

In [31]:
with uproot.open("data/SMHiggsToZZTo4L.root:Events") as tree:
    events_pt, events_eta, events_phi, events_charge = tree.arrays(
        ["Electron_pt", "Electron_eta", "Electron_phi", "Electron_charge"], how=tuple
    )

In [32]:
electrons = ak.to_backend(
    ak.zip({
        "pt": events_pt,
        "eta": events_eta,
        "phi": events_phi,
        "charge": events_charge,
    },
    with_name="Momentum4D",
), "cuda")

electrons

<Array [[], ..., [{pt: 48.07147, ...}, ...]] type='299973 * var * Momentum4...'>

With this backend, all mathematical computations are performed by CuPy (directly or through JIT compiled functions).

Below is a calculation of the mass of the heaviest dielectron in each event, using

$$\sqrt{2\,{p_T}_1\,{p_T}_2\left(\cosh(\eta_1 - \eta_2) - \cos(\phi_1 - \phi_2)\right)}$$

for the invariant mass.

In [33]:
e1, e2 = ak.unzip(ak.combinations(electrons, 2))
z_mass = np.sqrt(
    2*e1.pt*e2.pt * (np.cosh(e1.eta - e2.eta) - np.cos(e1.phi - e2.phi))
)
np.max(z_mass, axis=-1)

<Array [None, 76.752525, ..., 40.64742, 91.51784] type='299973 * ?float32'>

GPU-bound Awkward Arrays can also be iterated over in Numba-CUDA, which allows for imperative code. However, the output must be a non-ragged CuPy array.

In [34]:
ak.numba.register_and_check()

@nb.cuda.jit(extensions=[ak.numba.cuda])
def mass_of_heaviest_dielectron(electrons, out):
    thread_idx = nb.cuda.grid(1)
    if thread_idx < len(electrons):
        electrons_in_one_event = electrons[thread_idx]
        for i, e1 in enumerate(electrons_in_one_event):
            for e2 in electrons_in_one_event[i + 1:]:
                if e1.charge != e2.charge:
                    m = math.sqrt(
                        2*e1.pt*e2.pt * (math.cosh(e1.eta - e2.eta) - math.cos(e1.phi - e2.phi))
                    )
                    if m > out[thread_idx]:
                        out[thread_idx] = m

threads_per_block = 1024
blocks_per_grid = int(np.ceil(len(electrons) / 1024))

out = cp.zeros(len(electrons), dtype=np.float32)
mass_of_heaviest_dielectron[blocks_per_grid, threads_per_block](electrons, out)

out

array([ 0.      , 76.752525, 36.007294, ...,  0.      , 40.647415,
       91.51784 ], dtype=float32)

To make this a little more readable, let's rewrite it as a `__device__` function, which is a function that has a return value (doesn't have to overwrite an array) but can only be called from GPU-bound functions.

In [35]:
@nb.cuda.jit(extensions=[ak.numba.cuda], device=True)
def compute_mass(event):
    out = np.float32(0)
    for i, e1 in enumerate(event):
        for e2 in event[i + 1:]:
            if e1.charge != e2.charge:
                m = math.sqrt(
                    2*e1.pt*e2.pt * (math.cosh(e1.eta - e2.eta) - math.cos(e1.phi - e2.phi))
                )
                if m > out:
                    out = m
    return out

@nb.cuda.jit(extensions=[ak.numba.cuda])
def mass_of_heaviest_dielectron_2(events, out):
    thread_idx = nb.cuda.grid(1)
    if thread_idx < len(events):
        out[thread_idx] = compute_mass(events[thread_idx])

# same threads_per_block, blocks_per_grid

out = cp.zeros(len(electrons), dtype=np.float32)
mass_of_heaviest_dielectron_2[blocks_per_grid, threads_per_block](electrons, out)

out

array([ 0.      , 76.752525, 36.007294, ...,  0.      , 40.647415,
       91.51784 ], dtype=float32)

## Lesson 5 project: Histograms and Monte Carlo on GPUs

As described in the [intro](0-intro.md), navigate to the `notebooks` directory and open `lesson-5-project.ipynb`, then follow its instructions.